In [ ]:
import pandas as pd
import numpy as np
from pqdm.processes import pqdm
from scipy.stats import pearsonr

In [ ]:
"""
Helper to get_ligand_activity()
"""
def get_binary_sig(sig,background):
    return [1 if x in sig else 0 for x in background]

"""
Helper to call_ligand_activity()
"""
def get_ligand_activity(sig,potentials,background):
    binary_sig = get_binary_sig(sig,background)
    return pearsonr(binary_sig,potentials)[0]

"""
Helper to get_ligand_act_df()
"""
def call_ligand_activity(ligand,cond,sig):
    potentials = ltm.loc[:,ligand]
    la_score = get_ligand_activity(sig,potentials)
    return([cond,ligand,la_score])

"""
Calculate the ligand activity score of each condition's target genes.
    
Parameters:
-----------
cond_sig_d : dictionary
    Mapping between conditions (str) and target genes (list)

Returns:
--------
pd.DataFrame
    DataFrame with condition, ligand, and ligand activity score
"""
def get_ligand_act_df(cond_sig_d):
    args = []
    for cond,sig in cond_sig_d.items():
        for l in ltm.columns:
            args.append({'ligand':l,'condition':cond,'sig':sig})
    la_df = pqdm(args,call_ligand_activity,argument_type='kwargs',n_jobs=6)
    la_df = pd.DataFrame(la_df,columns=['condition','ligand','score'])
    
    return la_df

In [ ]:
# cosmx_panel.npy is a list of genes used in this study
# NicheNet_ligand_target_matrix.csv is the weight matrix describing associations between ligands and target genes
# described in Browaeys, Saelens, and Saeys (2019)
cosmx_panel = np.load('cosmx_panel.npy',allow_pickle=True)
ltm = pd.read_csv('NicheNet_ligand_target_matrix.csv',index_col=0)

targets_in_cos = list(set(ltm.index).intersection(set(cosmx_panel)))
ligands_in_cos = list(set(ltm.columns).intersection(set(cosmx_panel)))
ltm = ltm.loc[targets_in_cos,ligands_in_cos]

In [ ]:
# cond_sig_d is a mapping between a condition and target genes
# for example, the conditions can be every cell type in every zone at baseline
# and the genes are those that are differential between the cell type in the given zone and the rest of the zones
cond_sig_d = np.load('cond_sig_d.npy',allow_pickle=True).item()

In [ ]:
la_df = get_ligand_act_df(cond_sig_d)